In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference # mnist_inference.py

#### 1. 定义神经网络的参数。

In [ ]:
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 3001
MOVING_AVERAGE_DECAY = 0.99

#### 2. 定义训练的过程并保存TensorBoard的log文件。

In [ ]:
def train(mnist):
    #  输入数据的命名空间。
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = mnist_inference.inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)
    
    # 处理滑动平均的命名空间。
    with tf.name_scope("moving_average"):
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
        variables_averages_op = variable_averages.apply(tf.trainable_variables())
   
    # 计算损失函数的命名空间。
    with tf.name_scope("loss_function"):
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
        cross_entropy_mean = tf.reduce_mean(cross_entropy)
        loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    
    # 定义学习率、优化方法及每一轮执行训练的操作的命名空间。
    with tf.name_scope("train_step"):
        learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
            staircase=True)

        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        # 反向传播更新参数和更新每一个参数的滑动平均值
        # 为了一次完成更新参数和滑动平均参数,tf提供了`tf.control_dependencies` 和`tf.group`两种机制。
        # 等价于：train_op = tf.group(train_step, variables_averages_op)
        with tf.control_dependencies([train_step, variables_averages_op]):
            train_op = tf.no_op(name='train')
    
    writer = tf.summary.FileWriter("log/mnist_name_scope.log", tf.get_default_graph())
    
    # 训练模型。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # train steps.
        for i in range(TRAINING_STEPS):
            # Feed train data.
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            
            if i % 1000 == 0:
                # 配置运行时需要记录的信息。
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                # 运行时记录运行信息的proto。
                run_metadata = tf.RunMetadata()
                # train
                # 将配置信息和运行信息的porto传入运行过程，从而记录内存和时间消耗。
                _, loss_value, step = sess.run(
                    [train_op, loss, global_step], feed_dict={x: xs, y_: ys},
                    options=run_options, run_metadata=run_metadata)
                # 写入日志文件。
                writer.add_run_metadata(run_metadata=run_metadata, tag=("tag%d" % i), global_step=i)
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
            else:
                _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
                
    writer.close()

#### 3. 主函数。
#### run error:`Couldn't open CUDA library libcupti.so.9.0.`
* 解决方法
```shell
export LD_LIBRARY_PATH=/usr/local/cuda-9.0/extras/CUPTI/lib64:$LD_LIBRARY_PATH
```

In [ ]:
def main(argv=None): 
    mnist = input_data.read_data_sets("../0_datasets/MNIST_data/", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    main()